<a href="https://colab.research.google.com/github/Ivan-KKH/APAI4011_GP/blob/master/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install
!pip install -q transformers

In [2]:
#import libraries
import numpy as np
import pandas as pd
from transformers import pipeline
#from google.colab import files
import io
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import math
import warnings
warnings.filterwarnings("ignore")

2023-05-07 08:07:43.496196: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 08:07:43.539746: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 08:07:43.540314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 08:07:44.088742: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
#import CSV file
#uploaded = files.upload()


In [4]:
#read csv file
#df = pd.read_csv(io.BytesIO(uploaded['amazon.csv']))
df = pd.read_csv('amazon.csv')

#dfReview for review product name and product description
dfText = df[['review_title', 'review_content']]

#preprocess text data
for column in range(2):
    dfText.iloc[:,column] = dfText.iloc[:,column].apply(lambda x: x.lower())
    dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub(r"http\S+"," ",x)))
    dfText.iloc[:,column] = dfText.iloc[:,column].apply((lambda x: re.sub('[^a-zA-z0-9\s]'," ",x)))

#combine product_name and about_product
dfText['combined'] = dfText['review_title'] + ' ' + dfText['review_content']
dfText['combined']

0       satisfied charging is really fast value for mo...
1       a good braided cable for your type c device go...
2       good speed for earlier versions good product w...
3       good product good one nice really nice product...
4       as good as original decent good one for second...
                              ...                        
1460    received the product without spanner excellent...
1461    ok everything was good couldn t return bcoz i ...
1462    very good work but front melt after 2 month go...
1463    fan speed is slow good quality good product go...
1464    works perfect ok good product nice product  re...
Name: combined, Length: 1465, dtype: object

In [5]:
#preprocess numeric inputs
#rating count
df['rating_count'] = df['rating_count'].replace(',','',regex=True).astype('float64')
df['rating_count'] = df['rating_count'].fillna(0) #replace NaN values with 0

#actual price
df['actual_price'] = df['actual_price'].replace(',','',regex=True)
df['actual_price'] = df['actual_price'].replace('₹','',regex=True).astype('float64')
#discounted price
df['discounted_price'] = df['discounted_price'].replace(',','',regex=True)
df['discounted_price'] = df['discounted_price'].replace('₹','',regex=True).astype('float64')
#rating
df['rating'] = df['rating'].replace('|', '4.0').astype('float64')


Sentiment from review

In [6]:

#sentiment analysis
sentiment_pipeline = pipeline("sentiment-analysis")
sentiment = []
for review in tqdm(dfText['review_title']):
    #sentiment analysis using huggingface sentiment analyser

    review_sentiment = sentiment_pipeline(review[:512])
    #positive score if label='POSITIVE' else negative
    if review_sentiment[0]['label'] == 'POSITIVE':
        score = review_sentiment[0]['score']
    else:
        score = -1 * review_sentiment[0]['score']
    sentiment.append(score)
df['sentiment'] = sentiment

df[['review_title','sentiment']]


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
100%|██████████| 1465/1465 [00:20<00:00, 71.87it/s]


,review_title,sentiment
0,"Satisfied,Charging is really fast,Value for mo...",0.999341
1,"A Good Braided Cable for Your Type C Device,Go...",-0.610361
2,"Good speed for earlier versions,Good Product,W...",0.999560
3,"Good product,Good one,Nice,Really nice product...",0.995654
4,"As good as original,Decent,Good one for second...",0.999690
...,...,...
1460,"Received the product without spanner,Excellent...",0.976231
1461,"ok,everything was good couldn't return bcoz I ...",-0.954374
1462,"very good,Work but front melt after 2 month,Go...",-0.909178
1463,"Fan Speed is slow,Good quality,Good product,go...",0.997580


Linear Regression

In [7]:
#multiple linear regression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

scaler = StandardScaler()

#add review sentiment as an input variable
X = df[['actual_price', 'rating', 'rating_count', 'sentiment']]
y = df['discounted_price']

#normalise numeric inputs and output
X = scaler.fit_transform(X)
output_scaler = StandardScaler()
y = output_scaler.fit_transform(np.array(y).reshape(-1,1))

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 21, test_size = 0.3)

#fit multiple linear regression
regss = LinearRegression()
regss.fit(X_train, y_train)

#predict and print R2
y_pred = regss.predict(X_test)

print('R2 Score: ', r2_score(y_test, y_pred))

# print other metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))

R2 Score:  0.9229431867304001
Mean Squared Error:  0.0974400754107512


Linear SVM

In [8]:
#SVM
from sklearn.svm import SVR

#fit on SVM regressor with Gaussian RBF kernel
linear_svm = SVR(kernel = 'linear') #linear=0.927 or rbf=0.890
linear_svm.fit(X_train, y_train)

#predict and print R2
y_pred = linear_svm.predict(X_test)
print('R2 Score: ', r2_score(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))

R2 Score:  0.9279244307643423
Mean Squared Error:  0.09114118017083901


RBF SVM

In [9]:
#SVM
from sklearn.svm import SVR

#fit on SVM regressor with Gaussian RBF kernel
rbf_svm = SVR(kernel = 'rbf') #linear=0.927 or rbf=0.890
rbf_svm.fit(X_train, y_train)

#predict and print R2
y_pred = rbf_svm.predict(X_test)
print('R2 Score: ', r2_score(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))

R2 Score:  0.890197353163139
Mean Squared Error:  0.13884791927029783


Random Forest

In [10]:
# build a random forest regressor
from sklearn.ensemble import RandomForestRegressor

#fit on random forest regressor
rf = RandomForestRegressor(n_estimators = 100, random_state = 0)
rf.fit(X_train, y_train)

#predict and print R2
y_pred = rf.predict(X_test)
print('R2 Score: ', r2_score(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))




R2 Score:  0.9266792836479116
Mean Squared Error:  0.09271569673562345


We select Random Forest as our final model selection, we build a predictor function for demonstration.

In [15]:
import random



def predict(actual_price, rating, rating_count, review):
    #sentiment analysis
    rev_sent = sentiment_pipeline(review[:512])
    #positive score if label='POSITIVE' else negative
    if (rev_sent[0]['label'] == 'POSITIVE'):
        sentiment = review_sentiment[0]['score']
    else:
        sentiment = -1 * review_sentiment[0]['score']

    input = [[actual_price, rating, rating_count, sentiment]]
    Xnew = scaler.transform(input)

    # make a prediction
    ynew = rf.predict(Xnew)

    ynew = output_scaler.inverse_transform(ynew.reshape(1, -1))

    print('actual price: ', actual_price)
    print('rating: ', rating)
    print('rating count: ', rating_count)
    print('review: ', review)
    print("")
    print(f'suggested discounted price: {ynew[0][0]}')


rand_idx = random.randint(0, len(df))
actual_price = df.iloc[rand_idx]['actual_price']
rating = df.iloc[rand_idx]['rating']
rating_count = df.iloc[rand_idx]['rating_count']
review = df.iloc[rand_idx]['review_title']
actual_discounted_price = df.iloc[rand_idx]['discounted_price']

predict(actual_price, rating, rating_count, review)

print('actual discounted price: ', actual_discounted_price)

actual price:  499.0
rating:  3.3
rating count:  2804.0
review:  No TRRS to TRS converter in the box,Not suitable for DSLR camera,Good for beginners,OK,Don't buy,For beginners it's good,Average product,It is a good and useful mic for YouTube content maker

suggested discounted price: 185.66000000000076
actual discounted price:  199.0
